In [7]:
# Import the QICK drivers and auxiliary libraries
import matplotlib.pyplot as plt
import numpy as np
from qick import *
#%matplotlib inline

In [8]:
# Load bitstream with custom overlay
soc = QickSoc()
# Since we're running locally on the QICK, we don't need a separate QickConfig object.
# If running remotely, you could generate a QickConfig from the QickSoc:
#     soccfg = QickConfig(soc.get_cfg())
# or save the config to file, and load it later:
#     with open("qick_config.json", "w") as f:
#         f.write(soc.dump_cfg())
#     soccfg = QickConfig("qick_config.json")
soccfg = soc
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Global clocks (MHz): tProcessor 350.000, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 0, envelope memory 65536 samples
		DAC tile 0, ch 0, 32-bit DDS, fabric=614.400 MHz, fs=9830.400 MHz
	1:	axis_signal_gen_v6 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, ch 0, 32-bit DDS, fabric=614.400 MHz, fs=9830.400 MHz

	2 readout channels:
	0:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, ch 0, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 4, tProc input 0
	1:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, ch 1, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 5, tProc input 1

	2 DACs:
		DAC tile 0, ch 0 is DAC_B
		DAC tile 2, ch 0 is DAC_A

	2 ADCs:
		ADC tile 0, ch 0 is ADC_D
		ADC tile 0, ch 1 is ADC_C

	4 digital output pins (tProc output 7):
	0:	PMOD0_0
	1:	PMOD0_1
	2:	PMOD0_2
	3

In [54]:
class DoublePulseProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch0 = 0
        res_ch1 = 1

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch0"], nqz=1)
        self.declare_gen(ch=cfg["res_ch1"], nqz=1)
        
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
          #  for ch in cfg["ro_chs"]:
        self.declare_readout(ch=0, length=self.cfg["readout_length"],
                             freq=cfg["pulse_freq0"], gen_ch=0)

        self.declare_readout(ch=1, length=self.cfg["readout_length"],
                             freq=cfg["pulse_freq1"], gen_ch=1)
        
        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq0 = self.freq2reg(cfg["pulse_freq0"],gen_ch=res_ch0, ro_ch=0)
        freq1 = self.freq2reg(cfg["pulse_freq1"],gen_ch=res_ch1, ro_ch= 1)
        gain = cfg["pulse_gain"]

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            #sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch0, name="measure", sigma=10, length=10*5)
            self.add_gauss(ch=res_ch1, name="measure", sigma=10, length=10*5)
            
        if style == "const":
            self.default_pulse_registers(ch=res_ch0, style=style, freq=freq0, gain=gain, 
                                         length=cfg["length"])
            self.default_pulse_registers(ch=res_ch1, style=style, freq=freq1, gain=gain, 
                                         length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.default_pulse_registers(ch=res_ch0, style=style, freq=freq0, gain=gain, 
                                         waveform="measure", length=cfg["length"])
            self.default_pulse_registers(ch=res_ch1, style=style, freq=freq1, gain=gain, 
                                         waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.default_pulse_registers(ch=res_ch0, style=style, freq=freq0, gain=gain, 
                                         waveform="measure")
            self.default_pulse_registers(ch=res_ch1, style=style, freq=freq1, gain=gain, 
                                         waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        phase1 = self.deg2reg(self.cfg["res_phase0"], gen_ch=0)
        phase2 = self.deg2reg(self.cfg["res_phase1"], gen_ch=1)
        # fire a single trigger, but two pulses offset by 200 tProc clock ticks
        self.trigger(adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"])
        self.set_pulse_registers(ch=0, phase=phase1)
        self.pulse(ch=self.cfg["res_ch0"], t=0)
        self.set_pulse_registers(ch=1, phase=phase2)
        self.pulse(ch=self.cfg["res_ch1"], t=0)
        
        self.pulse(ch=self.cfg["res_ch0"], t=10)
        self.pulse(ch=self.cfg["res_ch1"], t=10)
        
        self.pulse(ch=self.cfg["res_ch0"], t=20)
        self.pulse(ch=self.cfg["res_ch1"], t=20)
        
        self.wait_all()
        self.sync_all(self.us2cycles(self.cfg["relax_delay"]))



In [55]:
config_1={"res_ch0":0, # --Fixed
        "ro_chs":[0], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase0":0, # --degrees
        "pulse_style": "const", # --Fixed
                
        #"sigma":10,
        "length": 5,
          
        "readout_length":250, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks
        
        "pulse_gain":30000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq0": 1000, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":100
        # Try varying soft_avgs from 1 to 200 averages
       }

config_2={"res_ch1":1, # --Fixed
        "ro_chs":[1], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase1":0, # --degrees
        "pulse_style": "const", # --Fixed
                
        #"sigma":10,
        "length": 5,
          
        "readout_length":250, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks
        
        "pulse_gain":30000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq1": 1000, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":100
        # Try varying soft_avgs from 1 to 200 averages
       }
config={**config_1,**config_2} #combine configs
prog =DoublePulseProgram(soccfg, config)
#adc1, adc2 = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)
#iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)

In [56]:
def send_double_pulse():
    adc1, adc2 = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)

In [57]:
send_double_pulse()

  0%|          | 0/100 [00:00<?, ?it/s]